In [2]:
from pickle import TRUE
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import datetime as dt
from datetime import datetime
import email.utils as eutils
import time
import streamlit as st
import eppy as eppy
from eppy import modeleditor
from eppy.modeleditor import IDF
from eppy.runner.run_functions import runIDFs
import PySimpleGUI as sg
from PIL import Image, ImageTk
import os
from eppy.results import readhtml # the eppy module with functions to read the html
from eppy.results import fasthtml
import subprocess
import os
from os import system

import pprint
pp = pprint.PrettyPrinter()

In [3]:
sg.theme('LightBlue2')
BAR_MAX = 24

tab0_layout =  [[sg.Text('Welcome')],
                [sg.Text('Please follow the steps below to test thermal resilience of the house')],
                [sg.Text('0. Click IMPORT TEMPLATE to creat a new blank IDF from template')],
                [sg.Text('1. Click IMPORT to import geometry from the base IDF and design weather data from DDY')],
                [sg.Text('2. Click CREATE to create an IDF from CSV Run List')],
                [sg.Text('3. Run parametric IDF using EP-Launch')],
                [sg.Text('4. Click COLLECT to compile results')],
                ]

tab1_layout =   [[sg.Text('Study Folder:', size =(15, 1)),sg.InputText('C:\\4StepResiliencev1-2', key='StudyFolder'), sg.FolderBrowse()],
                [sg.Text('Geometry IDF:', size =(15, 1)),sg.InputText('Select IDF with Geometry', key='GEO'), sg.FileBrowse()],
                [sg.Text('Template IDF:', size =(15, 1)),sg.InputText('C:\\4StepResiliencev1-2\\00_SF Base v1_noGeo.idf', key='template'), sg.FileBrowse()],
                [sg.Text('DDY:', size =(15, 1)),sg.InputText('Select DDY File', key='DDY'), sg.FileBrowse()],
                [sg.Text('IDD File Location:', size =(15, 1)),sg.InputText("C:\EnergyPlusV9-5-0\Energy+.idd", key='iddFile'), sg.FileBrowse()],
                [sg.Button('IMPORT TEMPLATE',  button_color=('white', 'orange'))],
                ]

tab2_layout =   [[sg.Text('File Name:', size =(20, 1)),sg.InputText('Name your file', key='BaseFileName')],
                [sg.Text('Occupancy:', size =(20, 1)),sg.InputText('4', key='Occupants')],
                [sg.Text('ICFA:', size =(20, 1)),sg.InputText('2128', key='ICFA')],
                [sg.Text('Partition to ICFA Ratio:', size =(20, 1)),sg.InputText('1.0', key='PartitionRatio')],
                [sg.Text('For plotting temperature and RH during outage:')],
                [sg.Text('Outage 1 Start Date:', size =(20, 1)),sg.InputText('2020-01-26 00:00:00', key='OS1')],
                [sg.Text('Outage 1 End Date:', size =(20, 1)),sg.InputText('2020-02-04 00:00:00', key='OE1')],
                [sg.Text('Outage 2 Start Date:', size =(20, 1)),sg.InputText('2020-07-19 00:00:00', key='OS2')],
                [sg.Text('Outage 2 End Date:', size =(20, 1)),sg.InputText('2020-07-30 00:00:00', key='OE2')],
                ]

layout1 = [
    # [sg.Image(r'C:\Users\amitc\Documents\GitHub\Phius-REVIVE\Project Program\4StepResilience\al_REVIVE_PILOT_logo.png')],
                [sg.TabGroup(
                [[sg.Tab('Start', tab0_layout,),
                sg.Tab('Project Settings', tab1_layout,),
                sg.Tab('Basic Input Data', tab2_layout,),]])],
                [sg.Text('COLLECT Progress:')],
                [sg.ProgressBar(BAR_MAX, orientation='h', size=(20,20), key='-PROG-')],
                [sg.Button('IMPORT'), sg.Button('CREATE'), sg.Button('COLLECT'), sg.Button('EXIT')]]  

window = sg.Window('4 Step Resilience Test v1.2',layout1, default_element_size=(125, 125), grab_anywhere=True)   